# pandas基础

In [120]:
import numpy as np
import pandas as pd

In [121]:
pd.__version__

'2.2.3'

## 文件的读取和写入

## 数据读取

In [122]:
# pandas 可以读取的文件格式有很多，这里主要介绍读取 csv, excel, txt 文件。
df_csv = pd.read_csv('data/my_csv.csv')
df_csv

,col1,col2,col3,col4,col5
0,2,a,1.4,apple,2020/1/1
1,3,b,3.4,banana,2020/1/2
2,6,c,2.5,orange,2020/1/5


In [123]:
df_txt = pd.read_table('data/my_table.txt')
df_txt

,col1,col2,col3,col4,col5
0,2,a,1.4,apple,2020/1/1
1,3,b,3.4,banana,2020/1/2
2,6,c,2.5,orange,2020/1/5


In [124]:
df_excel = pd.read_excel('data/my_excel.xlsx')
df_excel

,col1,col2,col3,col4,col5
0,2,a,1.4,apple,2020/1/1
1,3,b,3.4,banana,2020/1/2
2,6,c,2.5,orange,2020/1/5


* 这里有一些常用的公共参数，header=None 表示第一行不作为列名，index_col 表示把某一列或几列作为索
引，索引的内容将会在第三章进行详述，usecols 表示读取列的集合，默认读取所有的列，parse_dates 表示
需要转化为时间的列，关于时间序列的有关内容将在第十章讲解，nrows 表示读取的数据行数。上面这些参
数在上述的三个函数里都可以使用。

In [125]:
pd.read_table('data/my_table.txt', header=None)

,0,1,2,3,4
0,col1,col2,col3,col4,col5
1,2,a,1.4,apple,2020/1/1
2,3,b,3.4,banana,2020/1/2
3,6,c,2.5,orange,2020/1/5


In [126]:
 pd.read_csv('data/my_csv.csv', index_col=['col1', 'col2'])

,,col3,col4,col5
col1,col2,,,
2,a,1.4,apple,2020/1/1
3,b,3.4,banana,2020/1/2
6,c,2.5,orange,2020/1/5


In [127]:
pd.read_table('data/my_table.txt', usecols=['col1', 'col2'])

,col1,col2
0,2,a
1,3,b
2,6,c


In [128]:
pd.read_csv('data/my_csv.csv', parse_dates=['col5'])

,col1,col2,col3,col4,col5
0,2,a,1.4,apple,2020-01-01
1,3,b,3.4,banana,2020-01-02
2,6,c,2.5,orange,2020-01-05


In [129]:
pd.read_excel('data/my_excel.xlsx', nrows=2)

,col1,col2,col3,col4,col5
0,2,a,1.4,apple,2020/1/1
1,3,b,3.4,banana,2020/1/2


* 在读取 txt 文件时，经常遇到分隔符非空格的情况，read_table 有一个分割参数 sep ，它使得用户可以自定
义分割符号，进行 txt 数据的读取。例如，下面的读取的表以 |||| 为分割：

In [130]:
pd.read_table('data/my_table_special_sep.txt')

,col1 |||| col2
0,TS |||| This is an apple.
1,GQ |||| My name is Bob.
2,WT |||| Well done!


* 上面的结果显然不是理想的，这时可以使用 sep ，同时需要指定引擎为 python ：

In [131]:
pd.read_table('data/my_table_special_sep.txt',sep=' \|\|\|\| ', engine='python')

,col1,col2
0,TS,This is an apple.
1,GQ,My name is Bob.
2,WT,Well done!


* sep 是正则参数
* 在使用 read_table 的时候需要注意，参数 sep 中使用的是正则表达式，因此需要对 | 进行转义
变成 \| ，否则无法读取到正确的结果。有关正则表达式的基本内容可以参考第八章或者其他相
关资料。

## 数据写入

* 一般在数据写入中，最常用的操作是把 index 设置为 False ，特别当索引没有特殊意义的时候，这样的行为
能把索引在保存的时候去除。

In [132]:
df_csv.to_csv('data/my_csv_saved.csv', index=False)

In [133]:
df_excel.to_excel('data/my_excel_saved.xlsx', index=False)

* pandas 中没有定义 to_table 函数，但是 to_csv 可以保存为 txt 文件，并且允许自定义分隔符，常用制表符
\t 分割：

In [134]:
df_txt.to_csv('data/my_txt_saved.txt', sep='\t', index=False)

* 如果想要把表格快速转换为 markdown 和 latex 语言，可以使用 to_markdown 和 to_latex 函数，此处需要
安装 tabulate 包。

In [135]:
print(df_csv.to_markdown())

|    |   col1 | col2   |   col3 | col4   | col5     |
|---:|-------:|:-------|-------:|:-------|:---------|
|  0 |      2 | a      |    1.4 | apple  | 2020/1/1 |
|  1 |      3 | b      |    3.4 | banana | 2020/1/2 |
|  2 |      6 | c      |    2.5 | orange | 2020/1/5 |


In [136]:
print(df_csv.to_latex())

\begin{tabular}{lrlrll}
\toprule
 & col1 & col2 & col3 & col4 & col5 \\
\midrule
0 & 2 & a & 1.400000 & apple & 2020/1/1 \\
1 & 3 & b & 3.400000 & banana & 2020/1/2 \\
2 & 6 & c & 2.500000 & orange & 2020/1/5 \\
\bottomrule
\end{tabular}



## 基本数据结构

* pandas 中具有两种基本的数据存储结构，存储一维 values 的 Series 和存储二维 values 的 DataFrame ，在
这两种结构上定义了很多的属性和方法。

### Series

* Series 一般由四个部分组成，分别是序列的值 data 、索引 index 、存储类型 dtype 、序列的名字 name 。其
中，索引也可以指定它的名字，默认为空。

In [137]:
s = pd.Series(data = [100, 'a', {'dic1':5}],
              index = pd.Index(['id1', 20, 'third'], name='my_idx'),
              dtype = 'object',
              name = 'my_name')

In [138]:
s

my_idx
id1              100
20                 a
third    {'dic1': 5}
Name: my_name, dtype: object

#### object 类型

* object 代表了一种混合类型，正如上面的例子中存储了整数、字符串以及 Python 的字典数据结
构。此外，目前 pandas 把纯字符串序列也默认认为是一种 object 类型的序列，但它也可以用
string 类型存储，文本序列的内容会在第八章中讨论。
对于这些属性，可以通过 . 的方式来获取：

In [139]:
# 获取 Series 的值。
s.values

array([100, 'a', {'dic1': 5}], dtype=object)

In [140]:
# 获取 Series 的索引。
s.index

Index(['id1', 20, 'third'], dtype='object', name='my_idx')

In [141]:
# 获取 Series 的数据类型。
s.dtype

dtype('O')

In [142]:
# 获取 Series 的名称。
s.name

'my_name'

In [143]:
# 利用 .shape 可以获取序列的长度：
s.shape

(3,)

In [144]:
# 索引是 pandas 中最重要的概念之一，它将在第三章中被详细地讨论。如果想要取出单个索引对应的值，可
# 以通过 [index_item] 可以取出。
s['third']


{'dic1': 5}

### DataFrame

In [145]:
# DataFrame 在 Series 的基础上增加了列索引，一个数据框可以由二维的 data 与行列索引来构造：
data = [[1, 'a', 1.2], [2, 'b', 2.2], [3, 'c', 3.2]]

In [146]:
df = pd.DataFrame(data = data,
                  index = ['row_%d'%i for i in range(3)],
                  columns=['col_0', 'col_1', 'col_2'])

In [147]:
df

,col_0,col_1,col_2
row_0,1,a,1.2
row_1,2,b,2.2
row_2,3,c,3.2


In [148]:
# 但一般而言，更多的时候会采用从列索引名到数据的映射来构造数据框，同时再加上行索引：
df = pd.DataFrame(data = {'col_0': [1,2,3], 'col_1':list('abc'),
                          'col_2': [1.2, 2.2, 3.2]},
                  index = ['row_%d'%i for i in range(3)])

In [149]:
df

,col_0,col_1,col_2
row_0,1,a,1.2
row_1,2,b,2.2
row_2,3,c,3.2


In [150]:
# 由于这种映射关系，在 DataFrame 中可以用 [col_name] 与 [col_list] 来取出相应的列与由多个列组成的表，
# 结果分别为 Series 和 DataFrame ：
df['col_0']

row_0    1
row_1    2
row_2    3
Name: col_0, dtype: int64

In [151]:
df[['col_0', 'col_1']]

,col_0,col_1
row_0,1,a
row_1,2,b
row_2,3,c


In [152]:
# 与 Series 类似，在数据框中同样可以取出相应的属性：
# 以 numpy 数组的形式返回 DataFrame 中的数据。
df.values

array([[1, 'a', 1.2],
       [2, 'b', 2.2],
       [3, 'c', 3.2]], dtype=object)

In [153]:
# 返回 DataFrame 的行索引。可以是默认的整数索引，也可以是自定义的索引。
df.index

Index(['row_0', 'row_1', 'row_2'], dtype='object')

In [154]:
# 返回 DataFrame 的列索引，即列名。
df.columns

Index(['col_0', 'col_1', 'col_2'], dtype='object')

In [155]:
# 返回一个 Series，包含 DataFrame 每列的数据类型。
df.dtypes # 返回的是值为相应列数据类型的 Series

col_0      int64
col_1     object
col_2    float64
dtype: object

In [156]:
# 返回一个元组，包含 DataFrame 的行数和列数。
df.shape

(3, 3)

In [157]:
# 通过 .T 可以把 DataFrame 进行转置：
# 返回 DataFrame 的转置，即行和列互换。
df.T

,row_0,row_1,row_2
col_0,1,2,3
col_1,a,b,c
col_2,1.2,2.2,3.2


## 常用基本函数

* 为了进行举例说明，在接下来的部分和其余章节都将会使用一份 learn_pandas.csv 的虚拟数据集，它记录了
四所学校学生的体测个人信息。

In [158]:
df = pd.read_csv('data/learn_pandas.csv')

In [159]:
# 返回 DataFrame 的列索引，即列名。
df.columns

Index(['School', 'Grade', 'Name', 'Gender', 'Height', 'Weight', 'Transfer',
       'Test_Number', 'Test_Date', 'Time_Record'],
      dtype='object')

In [160]:
# 上述列名依次代表学校、年级、姓名、性别、身高、体重、是否为转系生、体测场次、测试时间、1000 米成
# 绩，本章只需使用其中的前七列。
df = df[df.columns[:7]]

### 汇总函数

In [161]:
# head, tail 函数分别表示返回表或者序列的前 n 行和后 n 行，其中 n 默认为 5：
# 它的主要功能是返回 DataFrame 的前几行,默认为 5
df.head(2)

,School,Grade,Name,Gender,Height,Weight,Transfer
0,A,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,B,Freshman,Changqiang You,Male,166.5,70.0,N


In [162]:
df.tail(3)

,School,Grade,Name,Gender,Height,Weight,Transfer
197,A,Senior,Chengqiang Chu,Female,153.9,45.0,N
198,A,Senior,Chengmei Shen,Male,175.3,71.0,N
199,D,Sophomore,Chunpeng Lv,Male,155.7,51.0,N


In [163]:
# 用于快速查看 DataFrame 的整体结构和数据情况
# Data columns：显示 DataFrame 中的列数。
# Column：各列的名称。
# Non-Null Count：每列的非空值数量，可用于检测缺失值。
# Dtype：每列的数据类型，如 int64、float64、object 等。
# memory usage：DataFrame 占用的内存大小。
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   School    200 non-null    object 
 1   Grade     200 non-null    object 
 2   Name      200 non-null    object 
 3   Gender    200 non-null    object 
 4   Height    183 non-null    float64
 5   Weight    189 non-null    float64
 6   Transfer  188 non-null    object 
dtypes: float64(2), object(5)
memory usage: 11.1+ KB


In [164]:
# 主要用于对 DataFrame 中的数值列进行统计分析，会计算这些列的各种统计指标，如计数、均值、标准差、最小值、四分位数和最大值等。
# 对于非数值列，该方法默认会跳过，但可以通过参数设置来包含它们。
df.describe()
# count：每列的非空值数量。
# mean：每列的平均值。
# std：每列的标准差，反映数据的离散程度。
# min：每列的最小值。
# 25%：每列数据的第一四分位数（下四分位数），表示有 25% 的数据小于该值。
# 50%：每列数据的中位数（第二四分位数），表示有 50% 的数据小于该值。
# 75%：每列数据的第三四分位数（上四分位数），表示有 75% 的数据小于该值。
# max：每列的最大值。

,Height,Weight
count,183.000000,189.000000
mean,163.218033,55.015873
std,8.608879,12.824294
min,145.400000,34.000000
25%,157.150000,46.000000
50%,161.900000,51.000000
75%,167.500000,65.000000
max,193.900000,89.000000


In [165]:
# info, describe 只能实现较少信息的展示，如果想要对一份数据集进行全面且有效的观察，特别是
# 在列较多的情况下，推荐使用 pandas-profiling 包，它将在第十一章被再次提到。

### 特征统计函数

* 在 Series 和 DataFrame 上定义了许多统计函数，最常见的是 sum, mean, median, var, std, max, min 。例
如，选出身高和体重列进行演示：

In [166]:
df_demo = df[['Height', 'Weight']]

In [167]:
# 默认会计算 DataFrame 中每列的平均值，并且会自动忽略缺失值（即 NaN）。仅对数值类型的列进行计算，非数值列会被跳过。
df_demo.mean()

Height    163.218033
Weight     55.015873
dtype: float64

In [168]:
# 默认会计算 DataFrame 中每列的最大值，并且会自动忽略缺失值（即 NaN）。对于数值列，会返回数值上的最大值；对于字符串列，会按照字典序返回最大的值。
df_demo.max()

Height    193.9
Weight     89.0
dtype: float64

In [169]:
# 此外，需要介绍的是 quantile, count, idxmax 这三个函数，它们分别返回的是分位数、非缺失值个数、最大
# 值对应的索引：
# 对于 DataFrame 对象，quantile() 方法默认会对每列数据分别计算指定的分位数。
# 分位数是将数据按从小到大排序后分割成若干等份的数值点
df_demo.quantile(0.75)

Height    167.5
Weight     65.0
Name: 0.75, dtype: float64

In [170]:
# 主要用于计算 DataFrame 中每列或每行的非空值数量
# df_demo.count() 方法有一个重要参数 axis，它可以控制计算非空值数量的方向：
# axis = 0：默认值，按列计算非空值数量，即对每列中的所有行数据统计非空值个数。
# axis = 1：按行计算非空值数量，也就是对每行中的所有列数据统计非空值个数。
df_demo.count()

Height    183
Weight    189
dtype: int64

In [171]:
# 计算 DataFrame 每列最大值对应的索引。
# 当某列（或行）存在多个最大值时，idxmax() 只会返回第一个最大值的索引。
# 对于非数值列，idxmax() 会按照字典序比较元素大小。
df_demo.idxmax() # idxmin 是对应的函数

Height    193
Weight      2
dtype: int64

In [172]:
# 上面这些所有的函数，由于操作后返回的是标量，所以又称为聚合函数，它们有一个公共参数 axis ，默认为
# 0 代表逐列聚合，如果设置为 1 则表示逐行聚合：
df_demo.mean(axis=1).head() # 在这个数据集上体重和身高的均值并没有意义

0    102.45
1    118.25
2    138.95
3     41.00
4    124.00
dtype: float64

### 唯一值函数

* 对序列使用 unique 和 nunique 可以分别得到其唯一值组成的列表和唯一值的个数：

In [173]:
# unique() 方法主要用于获取 Series 或 DataFrame 某列中唯一值组成的数组。也就是说，它会去除数据里的重复值，仅保留每个不同值的一个实例
df['School'].unique()
# unique() 方法返回的是 numpy 数组，而非 Series 对象。
# 它会保留值的原始顺序，即首次出现的顺序。

array(['A', 'B', 'C', 'D'], dtype=object)

In [174]:
# nunique() 方法用于计算 Series 或 DataFrame 某列中唯一值的数量。它返回的是一个表示唯一值个数的整数。
df['School'].nunique()
# nunique() 方法有一个重要参数 dropna，其默认值为 True，意味着在计算唯一值数量时会排除缺失值（NaN）；若将其设置为 False，则会把缺失值当作一个独特的值进行计数。

4

In [175]:
# value_counts 可以得到唯一值和其对应出现的频数：
# value_counts() 方法会返回一个新的 Series，其中索引是原 Series 中的唯一值，而对应的值是这些唯一值在原 Series 中出现的次数，并且默认按照频数从高到低排序。
df['School'].value_counts()

School
D    69
A    57
C    40
B    34
Name: count, dtype: int64

* 如果想要观察多个列组合的唯一值，可以使用 drop_duplicates 。其中的关键参数是 keep ，默认值 first 表
示每个组合保留第一次出现的所在行，last 表示保留最后一次出现的所在行，False 表示把所有重复组合所
在的行剔除。

In [176]:
df_demo = df[['Gender','Transfer','Name']]
df_demo.drop_duplicates(['Gender', 'Transfer'])
# drop_duplicates() 方法可以移除 DataFrame 中的重复行。默认情况下，它会考虑所有列来判断两行是否重复，只保留第一次出现的行。

,Gender,Transfer,Name
0,Female,N,Gaopeng Yang
1,Male,N,Changqiang You
12,Female,NaN,Peng You
21,Male,NaN,Xiaopeng Shen
36,Male,Y,Xiaojuan Qin
43,Female,Y,Gaoli Feng


In [177]:
df_demo.drop_duplicates(['Gender', 'Transfer'], keep='last')
# subset
# 该参数用于指定判断重复行时要考虑的列。可以传入列名列表，仅根据这些列的值来判断是否重复。
# keep
# 该参数用于指定保留重复行中的哪一个，有三个可选值：
# 'first'（默认值）：保留第一次出现的行，移除后续重复行。
# 'last'：保留最后一次出现的行，移除前面的重复行。
# False：移除所有重复行，不保留任何重复的行。

,Gender,Transfer,Name
147,Male,NaN,Juan You
150,Male,Y,Chengpeng You
169,Female,Y,Chengquan Qin
194,Female,NaN,Yanmei Qian
197,Female,N,Chengqiang Chu
199,Male,N,Chunpeng Lv


In [178]:
df_demo.drop_duplicates(['Name', 'Gender'],
                        keep=False).head() # 保留只出现过一次的性别和姓名组合

,Gender,Transfer,Name
0,Female,N,Gaopeng Yang
1,Male,N,Changqiang You
2,Male,N,Mei Sun
4,Male,N,Gaojuan You
5,Female,N,Xiaoli Qian


In [179]:
df['School'].drop_duplicates() # 在 Series 上也可以使用
# 对于 Series，drop_duplicates() 会移除其中的重复值，只保留第一次出现的值。

0    A
1    B
3    C
5    D
Name: School, dtype: object

* 此外，duplicated 和 drop_duplicates 的功能类似，但前者返回了是否为唯一值的布尔列表，其 keep 参数与
后者一致。其返回的序列，把重复元素设为 True ，否则为 False 。drop_duplicates 等价于把 duplicated 为
True 的对应行剔除。

In [180]:
df_demo.duplicated(['Gender', 'Transfer']).head()
# 对于 DataFrame，duplicated() 默认会考虑所有列，若某一行的所有列值与前面的某一行完全相同，就将该行标记为重复项。

0    False
1    False
2     True
3     True
4     True
dtype: bool

In [181]:
df['School'].duplicated().head() # 在 Series 上也可以使用
# 对于 Series，duplicated() 会检查每个元素，若某个元素不是第一次出现，就将其标记为重复项。

0    False
1    False
2     True
3    False
4     True
Name: School, dtype: bool

### 替换函数

* 一般而言，替换操作是针对某一个列进行的，因此下面的例子都以 Series 举例。pandas 中的替换函数可以
归纳为三类：映射替换、逻辑替换、数值替换。其中映射替换包含 replace 方法、第八章中的 str.replace 方
法以及第九章中的 cat.codes 方法，此处介绍 replace 的用法。
在 replace 中，可以通过字典构造，或者传入两个列表来进行替换：

In [182]:
# 将指定的值替换为其他值。
df['Gender'].replace({'Female':0, 'Male':1}).head()

/var/folders/94/hgkmp5gj0yz0kvgn1z2b1j7c0000gn/T/ipykernel_73128/4056047508.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Gender'].replace({'Female':0, 'Male':1}).head()


0    0
1    1
2    1
3    0
4    1
Name: Gender, dtype: int64

In [183]:
df['Gender'].replace(['Female', 'Male'], [0, 1]).head()

/var/folders/94/hgkmp5gj0yz0kvgn1z2b1j7c0000gn/T/ipykernel_73128/3987449977.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Gender'].replace(['Female', 'Male'], [0, 1]).head()


0    0
1    1
2    1
3    0
4    1
Name: Gender, dtype: int64

In [184]:
# 另外，replace 还有一种特殊的方向替换，指定 method 参数为 ffill 则为用前面一个最近的未被替换的值进行
# 替换，bfill 则使用后面最近的未被替换的值进行替换。从下面的例子可以看到，它们的结果是不同的：
s = pd.Series(['a', 1, 'b', 2, 1, 1, 'a'])

In [185]:
s.replace([1, 2], method='ffill')

/var/folders/94/hgkmp5gj0yz0kvgn1z2b1j7c0000gn/T/ipykernel_73128/3681488121.py:1: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  s.replace([1, 2], method='ffill')


0    a
1    a
2    b
3    b
4    b
5    b
6    a
dtype: object

In [186]:
s.replace([1, 2], method='bfill')

/var/folders/94/hgkmp5gj0yz0kvgn1z2b1j7c0000gn/T/ipykernel_73128/1140877935.py:1: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  s.replace([1, 2], method='bfill')


0    a
1    b
2    b
3    a
4    a
5    a
6    a
dtype: object

* 正则替换请使用 str.replace
* 虽然对于 replace 而言可以使用正则替换，但是当前版本下对于 string 类型的正则替换还存在
bug ，因此如有此需求，请选择 str.replace 进行替换操作，具体的方式将在第八章中讲解。

In [187]:
# 逻辑替换包括了 where 和 mask ，这两个函数是完全对称的：where 函数在传入条件为 False 的对应行进行
# 替换，而 mask 在传入条件为 True 的对应行进行替换，当不指定替换值时，替换为缺失值。
s = pd.Series([-1, 1.2345, 100, -50])
s.where(s<0)

0    -1.0
1     NaN
2     NaN
3   -50.0
dtype: float64

In [188]:
s.where(s<0, 100)

0     -1.0
1    100.0
2    100.0
3    -50.0
dtype: float64

In [189]:
s.mask(s<0)

0         NaN
1      1.2345
2    100.0000
3         NaN
dtype: float64

In [190]:
s.mask(s<0, -50)

0    -50.0000
1      1.2345
2    100.0000
3    -50.0000
dtype: float64

In [191]:
# 需要注意的是，传入的条件只需是与被调用的 Series 索引一致的布尔序列即可：
s_condition= pd.Series([True,False,False,True],index=s.index)

In [192]:
 s.mask(s_condition, -50)

0    -50.0000
1      1.2345
2    100.0000
3    -50.0000
dtype: float64

In [193]:
# 数值替换包含了 round, abs, clip 方法，它们分别表示取整、取绝对值和截断：
s = pd.Series([-1, 1.2345, 100, -50])
s.round(2)

0     -1.00
1      1.23
2    100.00
3    -50.00
dtype: float64

In [194]:
s.abs()

0      1.0000
1      1.2345
2    100.0000
3     50.0000
dtype: float64

In [195]:
 s.clip(0, 2) # 前两个数分别表示上下截断边界

0    0.0000
1    1.2345
2    2.0000
3    0.0000
dtype: float64

### 排序函数

* 排序共有两种方式，其一为值排序，其二为索引排序，对应的函数是 sort_values 和 sort_index 。
为了演示排序函数，下面先利用 set_index 方法把年级和姓名两列作为索引，多级索引的内容和索引设置的
方法将在第三章进行详细讲解。

In [196]:
df_demo = df[['Grade', 'Name', 'Height',
              'Weight']].set_index(['Grade','Name'])

In [197]:
# 对身高进行排序，默认参数 ascending=True 为升序：
df_demo.sort_values('Height').head()

,,Height,Weight
Grade,Name,,
Junior,Xiaoli Chu,145.4,34.0
Senior,Gaomei Lv,147.3,34.0
Sophomore,Peng Han,147.8,34.0
Senior,Changli Lv,148.7,41.0
Sophomore,Changjuan You,150.5,40.0


In [198]:
df_demo.sort_values('Height', ascending=False).head()

Height  Weight
Grade    Name                         
Senior   Xiaoqiang Qin   193.9    79.0
         Mei Sun         188.9    89.0
         Gaoli Zhao      186.5    83.0
Freshman Qiang Han       185.3    87.0
Senior   Qiang Zheng     183.9    87.0

In [199]:
# 在排序中，进场遇到多列排序的问题，比如在体重相同的情况下，对身高进行排序，并且保持身高降序排列，
# 体重升序排列：
df_demo.sort_values(['Weight','Height'],ascending=[True,False]).head()

,,Height,Weight
Grade,Name,,
Sophomore,Peng Han,147.8,34.0
Senior,Gaomei Lv,147.3,34.0
Junior,Xiaoli Chu,145.4,34.0
Sophomore,Qiang Zhou,150.5,36.0
Freshman,Yanqiang Xu,152.4,38.0


In [200]:
# 索引排序的用法和值排序完全一致，只不过元素的值在索引中，此时需要指定索引层的名字或者层号，用参
# 数 level 表示。另外，需要注意的是字符串的排列顺序由字母顺序决定。
df_demo.sort_index(level=['Grade','Name'],ascending=[True,False]).head()

Height  Weight
Grade    Name                         
Freshman Yanquan Wang    163.5    55.0
         Yanqiang Xu     152.4    38.0
         Yanqiang Feng   162.3    51.0
         Yanpeng Lv        NaN    65.0
         Yanli Zhang     165.1    52.0

### apply 方法

* apply 方法常用于 DataFrame 的行迭代或者列迭代，它的 axis 含义与第 2 小节中的统计聚合函数一致，apply
的参数往往是一个以序列为输入的函数。例如对于 .mean() ，使用 apply 可以如下地写出：

In [201]:
df_demo = df[['Height', 'Weight']]

In [202]:
def my_mean(x):
    res = x.mean()
    return res

In [203]:
df_demo.apply(my_mean)

Height    163.218033
Weight     55.015873
dtype: float64

In [204]:
# 同样的，可以利用 lambda 表达式使得书写简洁，这里的 x 就指代被调用的 df_demo 表中逐个输入的序列：
df_demo.apply(lambda x:x.mean())

Height    163.218033
Weight     55.015873
dtype: float64

In [205]:
# 若指定 axis=1 ，那么每次传入函数的就是行元素组成的 Series ，其结果与之前的逐行均值结果一致。
df_demo.apply(lambda x:x.mean(), axis=1).head()

0    102.45
1    118.25
2    138.95
3     41.00
4    124.00
dtype: float64

* 这里再举一个例子：mad 函数返回的是一个序列中偏离该序列均值的绝对值大小的均值，例如序列 1,3,7,10
中，均值为 5.25，每一个元素偏离的绝对值为 4.25,2.25,1.75,4.75，这个偏离序列的均值为 3.25。现在利用
apply 计算升高和体重的 mad 指标：

In [206]:
df_demo.apply(lambda x:(x-x.mean()).abs().mean())

Height     6.707229
Weight    10.391870
dtype: float64

In [207]:
# 这与使用内置的 mad 函数计算结果一致：
# df_demo.mad()

* 谨慎使用 apply
* 得益于传入自定义函数的处理，apply 的自由度很高，但这是以性能为代价的。一般而言，使用
pandas 的内置函数处理和 apply 来处理同一个任务，其速度会相差较多，因此只有在确实存在
自定义需求的情境下才考虑使用 apply 。

### 窗口对象

* pandas 中有 3 类窗口，分别是滑动窗口 rolling 、扩张窗口 expanding 以及指数加权窗口 ewm 。需要说明
的是，以日期偏置为窗口大小的滑动窗口将在第十章讨论，指数加权窗口见本章练习。

#### 滑窗对象

In [208]:
# 要使用滑窗函数，就必须先要对一个序列使用 .rolling 得到滑窗对象，其最重要的参数为窗口大小 window 。
s = pd.Series([1,2,3,4,5])
roller = s.rolling(window = 3)
roller

Rolling [window=3,center=False,axis=0,method=single]

* 在得到了滑窗对象后，能够使用相应的聚合函数进行计算，需要注意的是窗口包含当前行所在的元素，例如
在第四个位置进行均值运算时，应当计算 (2+3+4)/3，而不是 (1+2+3)/3：

In [209]:
roller.mean()

0    NaN
1    NaN
2    2.0
3    3.0
4    4.0
dtype: float64

In [210]:
roller.sum()

0     NaN
1     NaN
2     6.0
3     9.0
4    12.0
dtype: float64

In [211]:
# 对于滑动相关系数或滑动协方差的计算，可以如下写出：
s2 = pd.Series([1,2,6,16,30])
roller.cov(s2)

0     NaN
1     NaN
2     2.5
3     7.0
4    12.0
dtype: float64

In [212]:
roller.corr(s2)

0         NaN
1         NaN
2    0.944911
3    0.970725
4    0.995402
dtype: float64

In [213]:
# 此外，还支持使用 apply 传入自定义函数，其传入值是对应窗口的 Series ，例如上述的均值函数可以等效表示：
roller.apply(lambda x:x.mean())

0    NaN
1    NaN
2    2.0
3    3.0
4    4.0
dtype: float64

* shift, diff, pct_change 是一组类滑窗函数，它们的公共参数为 periods=n ，默认为 1，分别表示取向前第 n
个元素的值、与向前第 n 个元素做差（与 Numpy 中不同，后者表示 n 阶差分）、与向前第 n 个元素相比计
算增长率。这里的 n 可以为负，表示反方向的类似操作。

In [214]:
 s = pd.Series([1,3,6,10,15])

In [215]:
s.shift(2)

0    NaN
1    NaN
2    1.0
3    3.0
4    6.0
dtype: float64

In [216]:
 s.diff(3)

0     NaN
1     NaN
2     NaN
3     9.0
4    12.0
dtype: float64

In [217]:
s.pct_change()

0         NaN
1    2.000000
2    1.000000
3    0.666667
4    0.500000
dtype: float64

In [218]:
s.shift(-1)

0     3.0
1     6.0
2    10.0
3    15.0
4     NaN
dtype: float64

In [219]:
s.diff(-2)

0   -5.0
1   -7.0
2   -9.0
3    NaN
4    NaN
dtype: float64

In [220]:
# 将其视作类滑窗函数的原因是，它们的功能可以用窗口大小为 n+1 的 rolling 方法等价代替：
s.rolling(3).apply(lambda x:list(x)[0]) # s.shift(2)

0    NaN
1    NaN
2    1.0
3    3.0
4    6.0
dtype: float64

In [221]:
s.rolling(4).apply(lambda x:list(x)[-1]-list(x)[0]) # s.diff(3)

0     NaN
1     NaN
2     NaN
3     9.0
4    12.0
dtype: float64

In [222]:
def my_pct(x):
    L = list(x)
    return L[-1]/L[0]-1

In [223]:
s.rolling(2).apply(my_pct) # s.pct_change()

0         NaN
1    2.000000
2    1.000000
3    0.666667
4    0.500000
dtype: float64

#### 扩张窗口

* 扩张窗口又称累计窗口，可以理解为一个动态长度的窗口，其窗口的大小就是从序列开始处到具体操作的对
应位置，其使用的聚合函数会作用于这些逐步扩张的窗口上。具体地说，设序列为 a1, a2, a3, a4，则其每个
位置对应的窗口即 [a1]、[a1, a2]、[a1, a2, a3]、[a1, a2, a3, a4]。

In [224]:
s = pd.Series([1, 3, 6, 10])

In [225]:
s.expanding().mean()

0    1.000000
1    2.000000
2    3.333333
3    5.000000
dtype: float64